In [ ]:
import pandas as pd
import numpy as np
from math import pi
from bokeh.palettes import Spectral6, Category20c,Category10,Accent,Category20
from bokeh.palettes import Greys256,Inferno256,Magma256,Plasma256,Viridis256,Cividis256,Turbo256,linear_palette,Set3,Pastel1,Spectral5,Spectral6
from bokeh.io import show, output_file, output_notebook, push_notebook
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Slider
from bokeh.models.widgets import Div
from bokeh.transform import factor_cmap, cumsum
from bokeh.models.widgets import Paragraph,DataTable, TableColumn
from bokeh.models import ColumnDataSource,HoverTool
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models.layouts import LayoutDOM, Box, Row, Column, GridBox, Spacer, WidgetBox
from bokeh.layouts import widgetbox, column, row
import nltk
from bs4 import BeautifulSoup
import string
import re
from collections import Counter
from nltk import tokenize 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

df=pd.read_csv('train.tsv',delimiter='\t',encoding='utf-8')
df1=pd.read_csv('test.tsv',delimiter='\t',encoding='utf-8')
df.dropna(inplace=True)
def LableFunc(Sentiment):
    if Sentiment>=3:
        return 'Positive'
    elif Sentiment<=1:
        return 'Negative'
    else:
        return 'Neutral'
df['Label']=df['Sentiment'].apply(LableFunc)

stop_words=set(stopwords.words('english'))
lemma=WordNetLemmatizer()

def clean_Phrase(phrase_text):
    phrase_text=re.sub(r'http\S+','',phrase_text)                             #removing the url
    phrase_text=re.sub('[^a-zA-Z]','',phrase_text)                            #removing Numbers and punctuatino
    phrase_text=str(phrase_text).lower()                                      #Convert all characters into lowercase
    phrase_text=word_tokenize(phrase_text)                                    #Tokenization
    phrase_text=[item for item in phrase_text if item not in stop_words]      #Removing Stop Words
    phrase_text=[lemma.lemmatize(word=w,pos='v') for w in phrase_text]        #Lemmatization
    phrase_text=[i for i in phrase_text if len(i)>2]                          #Remove the words havig length <=2
    phrase_text=''.join(phrase_text)                                          #Converting list to string
    return phrase_text

df['clean_Phrase']=df['Phrase'].apply(clean_Phrase)
#df1['clean_Phrase']=df1['Phrase'].apply(clean_Phrase)

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
text = df['clean_Phrase'].values
words = '' 
for val in df['clean_Phrase']:   
     val = str(val) 
     tokens = val.split('_')  
     words += " ".join(tokens)+" "
wordcloud = WordCloud(
    width = 300,
    height = 300,
    background_color = 'white',
    min_font_size=4,
    stopwords = STOPWORDS).generate(str(words))
fig = plt.figure(
    figsize = (4, 4),
    facecolor=None)
plt.imshow(wordcloud, interpolation = 'spline36')
plt.axis('off')
plt.tight_layout(pad=0)
#plt.show()

wordcloud.to_file("wc.png")

output_file("NLP_Dashboard_slider.html")
tools="pan,wheel_zoom,box_zoom,reset,save,box_select,hover"

x = Counter({
    'train': 156060,
    'test': 66292,
    'dev': 0
})
d = pd.DataFrame.from_dict(dict(x), orient='index').reset_index().rename(index=str, columns={0:'value', 'index':'data'})
d['percent'] = d['value'] / sum(x.values()) * 100
d['angle'] = d['value'] / sum(x.values()) * 2*pi
d['color'] = Accent[len(x)]
#z=110*(data['value']/data['value'].sum())
p2 = figure(plot_height=400,plot_width=400, title="Count of train and test",
           tools="hover", tooltips="@data:@value", x_range=(-0.5, 0.5))
p2.annular_wedge(x=0, y=1, inner_radius=0.10, outer_radius=0.25, direction="anticlock",
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', source=d)

#labels = LabelSet(x=0, y=1, text='value',
        #angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')

#p2.add_layout(labels)
p2.axis.axis_label=None
p2.axis.visible=False
p2.grid.grid_line_color = None

x=df['Sentiment'].value_counts()
data = pd.Series(x).reset_index(name='Count').rename(columns={'index':'Sentiment'})
#p4 = figure(tools="pan,wheel_zoom,box_zoom,reset")
p4 = figure(plot_width=400, plot_height=400,x_axis_label="Count of each sentiments", y_axis_label="Count",title="Count of Each Sentiments", tools=tools,toolbar_location="right" ,tooltips=[("Sentiment","@x"),("Count","@top")])
p4.vbar(x=range(5), top=data['Count'], width=0.7,color=Spectral5,line_color="black")
p4.y_range.start =0
#p4.y_range.end = 100000
#p4.xaxis.major_label_orientation = pi/4
p4.grid.grid_line_color = None

p3 = figure(plot_width=400, plot_height=400, title="WORDCOUNT")
p3.image_url(url=['wc.png'], x=0, y=0, w=0.8, h=0.8,anchor="bottom_left")
p3.xaxis.visible = None
p3.yaxis.visible = None


x=df['Sentiment'].value_counts()
data = pd.Series(x).reset_index(name='Count').rename(columns={'index':'Sentiment'})
#data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['Count']/data['Count'].sum() * 2*pi
data['percent'] = data['Count'] /data['Count'].sum() * 100
data['color'] = Category20[len(x)]
p5 = figure(plot_height=400,plot_width=400, title="Percentage of each sentiments",
           tools="hover", tooltips="@Sentiment: @percent{0.2f} %", x_range=(-0.5, 1.0))
p5.wedge(x=0.3, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', source=data)
p5.axis.axis_label=None
p5.axis.visible=False
p5.grid.grid_line_color = None

df0=df[(df.Sentiment ==0)]
df0.sort_values(by=['Phrase'], inplace=True,ascending=False)
#df0.head(10)
x=df0['clean_Phrase'].value_counts()
data0 = pd.Series(x).reset_index(name='Count').rename(columns={'index':'Sentiment0'})
df0=data0.head(30)
#df0.sort_values(by=['Count'], inplace=True,ascending=False)

df1=df[(df.Sentiment ==1)]
df1.sort_values(by=['Phrase'], inplace=True,ascending=False)
#df1.head(10)
x=df1['clean_Phrase'].value_counts()
data1 = pd.Series(x).reset_index(name='Count').rename(columns={'index':'Sentiment1'})
df1=data1.head(30)
#df0.sort_values(by=['Count'], inplace=True,ascending=False)

df2=df[(df.Sentiment ==2)]
df2.sort_values(by=['Phrase'], inplace=True,ascending=False)
#df2.head(10)
x=df2['clean_Phrase'].value_counts()
data2 = pd.Series(x).reset_index(name='Count').rename(columns={'index':'Sentiment2'})
df2=data2.head(30)
#df0.sort_values(by=['Count'], inplace=True,ascending=False)

df3=df[(df.Sentiment ==3)]
df3.sort_values(by=['Phrase'], inplace=True,ascending=False)
#df3.head(10)
x=df3['clean_Phrase'].value_counts()
data3 = pd.Series(x).reset_index(name='Count').rename(columns={'index':'Sentiment3'})
df3=data3.head(30)
#df0.sort_values(by=['Count'], inplace=True,ascending=False)

df4=df[(df.Sentiment ==4)]
df4.sort_values(by=['Phrase'], inplace=True,ascending=False)
#df4.head(10)
x=df4['clean_Phrase'].value_counts()
data4 = pd.Series(x).reset_index(name='Count').rename(columns={'index':'Sentiment4'})
df4=data4.head(30)
#df0.sort_values(by=['Count'], inplace=True,ascending=False)

def changeArea(attr, old, new):
    a=slider.value
    b=slider1.value
    if(a==0):
    	source.data = df0.head(b)
    if(a==1):
    	source.data = df1.head(b)
    if(a==2):
    	source.data = df2.head(b)
    if(a==3):
    	source.data = df3.head(b)
    if(a==4):
    	source.data = df4.head(b)

source = ColumnDataSource(data=df0.head(5))
columns = [
    TableColumn(field="clean_Phrase", title='CleanPhrase'),
    TableColumn(field="Count", title='Count')
    ]
p6= DataTable(source=source, columns=columns, width=600, height=400)
slider = Slider(start=0, end=4, value=0, step=1, title="Sentiment Slider",bar_color="green")
slider.on_change('value', changeArea)
slider1 = Slider(start=5, end=30, value=5, step=5, title="Top Slider",bar_color="red")
slider1.on_change('value', changeArea)

pre=Div(text=""" <div><h3><strong><center> Sentimental Analysis</center></strong><h3></div>""",
        align='center',style={'color':'firebrick','font-size':'25px','font-family':'Helvetica'})
col1 = row(p2,p4,p3)
col2 = row(slider,slider1)
col3 = row(p6,p5)
from bokeh.io import curdoc
#layout = column(widgetbox(slider,slider1), p6)
layout=column(pre,col1,col2,col3)
curdoc().add_root(layout)

